In [1]:
import glob
import os
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATA_PATH="data/"

In [4]:
glob.glob(DATA_PATH + "*")

['data/Scraping Acórdãos-20250111T184338Z-023.zip',
 'data/pr-irpf-2019.pdf',
 'data/normas_full.json',
 'data/llama_deduplicated_reference_titles_20250210_fixed.pkl',
 'data/pr-irpf-2016.pdf',
 'data/extracted_questions_20250202_parcial_02.pkl',
 'data/extracted_questions_20250202_parcial_01.pkl',
 'data/pr-irpf-2017.pdf',
 'data/extracted_questions_20250207_final.pkl',
 'data/llama_deduplicated_reference_titles_20250210.pkl',
 'data/pr-irpf-2021.pdf',
 'data/extracted_questions_20250203_parcial_01.pkl',
 'data/extracted_document_matches_20250210.pkl',
 'data/extracted_questions_20250202_parcial_03.pkl',
 'data/llama_deduplicated_reference_titles_20250205_02.pkl',
 'data/chunks_normas_new.pkl',
 'data/P&R IRPF 2023 - v1.1 - 04042023.pdf',
 'data/extracted_questions_20241029.pkl',
 'data/extracted_questions_with_embedded_references_20250212_before_formatting.pkl',
 'data/pr-irpf-2020.pdf',
 'data/pr-irpf-2018.pdf',
 'data/Scraping Normas-20250111T184306Z-001.zip',
 'data/PR-IRPF-2022-v

In [6]:
with open(DATA_PATH + "extracted_questions_with_embedded_references_20250212_before_formatting.pkl", "rb") as input_file:
    questions = pickle.load(input_file)

In [9]:
questions[0]

{'question_number': '001',
 'question_summary': 'OBRIGATORIEDADE',
 'question_text': 'Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024, ano-calendário de 2023?',
 'answer': ['Está obrigada a apresentar a Declaração de Ajuste Anual (DAA) referente ao exercício de 2024, a pessoa',
  'física residente no Brasil que, no ano-calendário de 2023:',
  '1 - recebeu rendimentos tributáveis, sujeitos ao ajuste na declaração, cuja soma foi superior a R$ 30.639,90',
  '(trinta mil, seiscentos e trinta e nove reais e noventa centavos);',
  '2 - recebeu rendimentos isentos, não tributáveis ou tributados exclusivamente na fonte, cuja soma foi superior',
  'a R$ 200.000,00 (duzentos mil reais);',
  '3 - obteve, em qualquer mês, ganho de capital na alienação de bens ou direitos sujeito à incidência do imposto;',
  '4 - realizou operações de alienação em bolsas de valores, de mercadorias, de futuros e assemelhadas:',
  'a) cuja soma foi superior a R$ 40.000,00 (q

## Format the extracted embedded legal references using LLM

In [117]:
groq_llama33_70b_interface = get_llm_interface()

In [33]:
processing_start = time.time()

total_prompt_tokens = 0
total_completion_tokens = 0

for i, which_question in enumerate(questions[263:]):
    print("\n*****************************************")
    print(f"Processing question {i}\n")
    print("*****************************************\n")

    cleaned_references = "; ".join(which_question['embedded_references']).replace("\n", " ")
    
    if len(cleaned_references) > 0:
        result = legal_references_formatting(groq_llama33_70b_interface, cleaned_references)

        which_question['formatted_embedded_references'] = result['referências']

        total_prompt_tokens += result['prompt_tokens']
        total_completion_tokens += result['completion_tokens']    
    else:
        print(">>> NO LEGAL REFERENCES TO PROCESS")

        which_question['formatted_embedded_references'] = None
    
processing_end = time.time()


*****************************************
Processing question 0

*****************************************


Lei nº 12.794, de 2 de abril de 2013, art. 18; Lei nº 7.713, de 22 de dezembro de 1988, art. 9º, inciso I; Lei nº 7.713/88, art. 3º, § 1º; Decreto nº 9.580/18, arts. 4º e 46 do Anexo; Decreto-Lei nº 1.301/73, arts. 3º, caput e § 1º; e 4º; Instrução Normativa SRF nº 118, de 28 de dezembro de 2000; Ação Direta de Inconstitucionalidade (ADI) nº 5.422, do Supremo Tribunal Federal
[{'role': 'system', 'content': 'Leia a lista de referências jurídicas e processe as informações existentes, separando de maneira estruturada o título da referência e as informações de artigos e anexos que forem explicitamente citadas. Se nenhum artigo ou anexo for citado, deixe a lista correspondente vazia. Sua resposta deve ser apenas o JSON no formato a seguir, nenhum comentário a mais: {"referências":[{ "título": "<nome-completo-da-lei-ou-documento-jurídico-incluindo-instrumento-aprovação>", "artigos": 

In [34]:
print(f"Total time to process the legal references: {processing_end - processing_start}")
print(f"total_prompt_tokens={total_prompt_tokens}, total_completion_tokens={total_completion_tokens}")

Total time to process the legal references: 125.25899505615234
total_prompt_tokens=41420, total_completion_tokens=16923


### Save the final result

In [35]:
with open(DATA_PATH + "extracted_questions_with_embedded_references_20250212_final.pkl", "wb") as output_file:
    pickle.dump(questions, output_file, pickle.HIGHEST_PROTOCOL)

## Now work to deduplicate the extracted embedded legal references

In [116]:
from references_normalization import *

### Consolidate a list of unique legal document extracted embedded references, even if somehow duplicated

In [37]:
unique_references, questions_without_references = return_unique_references(questions, which_field='formatted_embedded_references')

{'question_number': '002', 'question_summary': 'PESSOA FÍSICA DESOBRIGADA', 'question_text': 'Pessoa física desobrigada pode apresentar a Declaração de Ajuste Anual (DAA)?', 'answer': ['Sim. A pessoa física, ainda que desobrigada, pode apresentar a Declaração de Ajuste Anual (DAA), sendo', 'vedado a um mesmo contribuinte constar simultaneamente em mais de uma Declaração de Ajuste Anual, seja', 'como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de', '2023.', '(Instrução Normativa RFB nº 2.178, de 5 de março de 2024, art. 2º, § 2º)', 'Consulte as perguntas 001 e 174'], 'answer_cleaned': ['Sim. A pessoa física, ainda que desobrigada, pode apresentar a Declaração de Ajuste Anual (DAA), sendo', 'vedado a um mesmo contribuinte constar simultaneamente em mais de uma Declaração de Ajuste Anual, seja', 'como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de', '2023.', '', ''], 'references': ['I

In [38]:
len(questions_without_references)

524

In [39]:
sorted_unique_references = np.sort(list(unique_references.keys()))

In [40]:
sorted_unique_references.shape

(212,)

### Group the references by initial letter before sending to LLM for deduplication

In [42]:
from itertools import groupby

In [43]:
unique_references_by_initial_letter = {key: list(group) for key, group in groupby(sorted_unique_references, key=lambda x: x[0])}

In [44]:
unique_references_by_initial_letter.keys()

dict_keys(['A', 'C', 'D', 'E', 'I', 'L', 'M', 'N', 'P', 'R', 'S', 'e'])

In [45]:
group_counts = {}

for key, value in unique_references_by_initial_letter.items():
    group_counts[key] = len(value)

print(group_counts)

{'A': 28, 'C': 10, 'D': 30, 'E': 1, 'I': 23, 'L': 83, 'M': 6, 'N': 1, 'P': 13, 'R': 8, 'S': 8, 'e': 1}


### Deduplicate the references using LLM

In [118]:
processing_start = time.time()

llama_deduplicated_reference_titles = []
total_prompt_tokens = 0
total_completion_tokens = 0

for initial_letter, references in unique_references_by_initial_letter.items():
    print("\n*****************************************")
    print(f"Processing titles starting with {initial_letter}\n")
    print("*****************************************\n")

    if len(references) > 1:    
        result = legal_refereces_titles_deduplication(groq_llama33_70b_interface, references)
    
        print(f">> Original titles count={len(references)}; deduplicated titles count={len(result['deduplicated-references'])}")
    
        llama_deduplicated_reference_titles += result["deduplicated-references"]
    
        total_prompt_tokens += result['prompt_tokens']
        total_completion_tokens += result['completion_tokens']
    else:
        print(f">> Single document directly added to the list")
        llama_deduplicated_reference_titles += references

processing_end = time.time()


*****************************************
Processing titles starting with A

*****************************************



Titles deduplication. 28 elements


Lista de documentos:
Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha
Acórdão do RE nº 855.091/RS (Tema 808)
Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008
Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005
Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005
Ato Declaratório Interpretativo SRF nº 5, de 27 de abril de 2005
Ato Declaratório Interpretativo SRF nº 7, de 25 de março de 2004
Ato Declaratório Normativo CST nº 25, de 1989
Ato Declaratório Normativo Cosit nº 24, de 14 de setembro de 1999
Ato Declaratório PGFN Nº 2, de 10 de março de 2016
Ato Declaratório PGFN nº 1, de 12 de março de 2018
Ato Declaratório PGFN nº 1, de 18 de fevereiro de 2005
Ato Declaratório PGFN nº 1, de 2 de janeiro de 2014
Ato Declar

In [119]:
print(f"Total time to process the legal references: {processing_end - processing_start}")
print(f"total_prompt_tokens={total_prompt_tokens}, total_completion_tokens={total_completion_tokens}")

Total time to process the legal references: 19.768993139266968
total_prompt_tokens=6301, total_completion_tokens=3866


In [120]:
llama_deduplicated_reference_titles

['Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha',
 'Acórdão do RE nº 855.091/RS (Tema 808)',
 'Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008',
 'Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005',
 'Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005',
 'Ato Declaratório Interpretativo SRF nº 5, de 27 de abril de 2005',
 'Ato Declaratório Interpretativo SRF nº 7, de 25 de março de 2004',
 'Ato Declaratório Normativo CST nº 25, de 1989',
 'Ato Declaratório Normativo Cosit nº 24, de 14 de setembro de 1999',
 'Ato Declaratório PGFN Nº 1, de 12 de março de 2018',
 'Ato Declaratório PGFN Nº 1, de 18 de fevereiro de 2005',
 'Ato Declaratório PGFN Nº 1, de 2 de janeiro de 2014',
 'Ato Declaratório PGFN Nº 1, de 27 de março de 2009',
 'Ato Declaratório PGFN Nº 2, de 10 de março de 2016',
 'Ato Declaratório PGFN nº 4, de 12 de agosto de 2002',
 'Ato Declaratório PGFN nº 

In [121]:
len(llama_deduplicated_reference_titles)

175

#### Manual adjusts to remove wrongly recognized references

In [122]:
llama_deduplicated_reference_titles = llama_deduplicated_reference_titles[:-1]

In [123]:
llama_deduplicated_reference_titles

['Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha',
 'Acórdão do RE nº 855.091/RS (Tema 808)',
 'Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008',
 'Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005',
 'Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005',
 'Ato Declaratório Interpretativo SRF nº 5, de 27 de abril de 2005',
 'Ato Declaratório Interpretativo SRF nº 7, de 25 de março de 2004',
 'Ato Declaratório Normativo CST nº 25, de 1989',
 'Ato Declaratório Normativo Cosit nº 24, de 14 de setembro de 1999',
 'Ato Declaratório PGFN Nº 1, de 12 de março de 2018',
 'Ato Declaratório PGFN Nº 1, de 18 de fevereiro de 2005',
 'Ato Declaratório PGFN Nº 1, de 2 de janeiro de 2014',
 'Ato Declaratório PGFN Nº 1, de 27 de março de 2009',
 'Ato Declaratório PGFN Nº 2, de 10 de março de 2016',
 'Ato Declaratório PGFN nº 4, de 12 de agosto de 2002',
 'Ato Declaratório PGFN nº 

In [124]:
len(llama_deduplicated_reference_titles)

174

#### Save the final list

In [125]:
with open(DATA_PATH + "llama_deduplicated_embedded_reference_titles_20250213_fixed.pkl", "wb") as output_file:
    pickle.dump(llama_deduplicated_reference_titles, output_file, pickle.HIGHEST_PROTOCOL)

## Finally merge this new list with the annotated references list

In [77]:
with open(DATA_PATH + "llama_deduplicated_reference_titles_20250210_fixed.pkl", "rb") as input_file:
    annotated_references_list = pickle.load(input_file)

In [126]:
len(annotated_references_list)

403

In [127]:
common_docs = np.intersect1d(llama_deduplicated_reference_titles, annotated_references_list)

In [128]:
common_docs

array(['Ato Declaratório PGFN nº 5, de 3 de maio de 2016',
       'Ato Declaratório PGFN nº 9, de 20 de dezembro de 2011',
       'Decreto nº 27.784, de 16 de fevereiro de 1950',
       'Decreto nº 52.288, de 24 de julho de 1963',
       'Decreto nº 59.308, de 23 de setembro de 1966',
       'Decreto nº 63.151, de 22 de agosto de 1968',
       'Decreto nº 9.580, de 22 de novembro de 2018',
       'Decreto-Lei nº 167, de 14 de fevereiro de 1967',
       'Decreto-Lei nº 5.844, de 23 de setembro de 1943',
       'Instrução Normativa RFB nº 1.343, de 5 de abril de 2013',
       'Instrução Normativa RFB nº 1.500, de 29 de outubro de 2014',
       'Instrução Normativa RFB nº 1.585, de 31 de agosto de 2015',
       'Instrução Normativa RFB nº 1.888, de 3 de maio de 2019',
       'Instrução Normativa RFB nº 2.066, de 24 de fevereiro de 2022',
       'Instrução Normativa RFB nº 2.172, de 9 de janeiro de 2024',
       'Instrução Normativa RFB nº 2.178, de 5 de março de 2024',
       'Instrução N

In [129]:
common_docs.shape

(69,)

In [130]:
sorted_merged_references = np.sort(np.union1d(llama_deduplicated_reference_titles, annotated_references_list))

In [131]:
sorted_merged_references.shape

(507,)

In [132]:
for item in list(sorted_merged_references):
    print(item)

AD PGFN nº 3 de 18 de setembro de 2008
ADI RFB nº 12, de 2016
Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha
Acórdão do RE nº 855.091/RS (Tema 808)
Anexo ao Decreto nº 361, de 10 de dezembro de 1991
Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008
Ato Declaratório Cosar nº 47, de 27 de novembro de 2000
Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019
Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024
Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005
Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016
Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007
Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016
Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005
Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004
Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005
Ato Declaratório Int

In [133]:
final_deduplicated_references_list, processing_stats = deduplicate_legal_references_list(sorted_merged_references, groq_llama33_70b_interface)

{'A': 63, 'C': 13, 'D': 52, 'E': 1, 'I': 49, 'L': 120, 'M': 9, 'N': 6, 'P': 55, 'R': 7, 'S': 132}

*****************************************
Processing titles starting with A

*****************************************



Titles deduplication. 63 elements


Lista de documentos:
AD PGFN nº 3 de 18 de setembro de 2008
ADI RFB nº 12, de 2016
Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha
Acórdão do RE nº 855.091/RS (Tema 808)
Anexo ao Decreto nº 361, de 10 de dezembro de 1991
Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008
Ato Declaratório Cosar nº 47, de 27 de novembro de 2000
Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019
Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024
Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005
Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016
Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007
Ato Decla

In [134]:
len(final_deduplicated_references_list)

483

In [135]:
final_deduplicated_references_list = np.sort(final_deduplicated_references_list)

In [136]:
for item in list(final_deduplicated_references_list):
    print(item)

AD PGFN nº 3 de 18 de setembro de 2008
ADI RFB nº 12, de 2016
Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha
Acórdão do RE nº 855.091/RS (Tema 808)
Anexo ao Decreto nº 361, de 10 de dezembro de 1991
Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008
Ato Declaratório Cosar nº 47, de 27 de novembro de 2000
Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019
Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024
Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005
Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016
Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007
Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016
Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005
Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004
Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005
Ato Declaratório Int

#### Save the merge result

In [137]:
with open(DATA_PATH + "llama_deduplicated_embedded_and_annoatated_reference_titles_20250213_before_fixes.pkl", "wb") as output_file:
    pickle.dump(final_deduplicated_references_list, output_file, pickle.HIGHEST_PROTOCOL)

In [54]:
r1 = '{"deduplicated-references":["AD PGFN nº 3 de 18 de setembro de 2008","ADI RFB nº 12, de 2016","Acordo sobre o Novo Banco de Desenvolvimento","Anexo ao Decreto nº 361, de 10 de dezembro de 1991","Ato Declaratório Cosar nº 47, de 27 de novembro de 2000","Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019","Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024","Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005","Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016","Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007","Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016","Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004","Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007","Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004","Ato Declaratório Interpretativo SRF nº 26, de 26 de dezembro de 2003","Ato Declaratório Interpretativo SRF nº 8, de 23 de abril de 2003","Ato Declaratório Interpretativo nº 16, de 22 de dezembro de 2005","Ato Declaratório Normativo CST nº 10, de 1991","Ato Declaratório Normativo CST nº 11, de 1978","Ato Declaratório Normativo CST nº 16, de 27 de julho de 1979","Ato Declaratório Normativo CST nº 25, de 13 de dezembro de 1989","Ato Declaratório Normativo CST nº 29, de 25 de junho de 1986","Ato Declaratório Normativo Cosit nº 19, de 25 de outubro de 2000","Ato Declaratório Normativo Cosit nº 20, de 1989","Ato Declaratório Normativo Cosit nº 28, de 27 de dezembro de 1996","Ato Declaratório Normativo Cosit nº 7, de 13 de janeiro de 1997","Ato Declaratório Normativo Cosit nº 9, de 1º de abril de 1999","Ato Declaratório PGFN nº 1, de 12 de março de 2018","Ato Declaratório PGFN nº 3, de 30 de março de 2016","Ato Declaratório PGFN nº 4, de 16 de novembro de 2006","Ato Declaratório PGFN nº 5, de 3 de maio de 2016","Ato Declaratório PGFN nº 9, de 20 de dezembro de 2011","Ato Declaratório SRF nº 22, de 30 de abril de 1997","Ato Declaratório SRF nº 28, de 26 de abril de 2000","Ato Declaratório SRF nº 3, de 7 de janeiro de 1999","Ato Declaratório SRF nº 48, de 27 de junho de 2000","Ato Declaratório SRF nº 84, de 6 de outubro de 1999","Ato Declaratório SRF nº 95, de 26 de novembro de 1999","Ato Declaratório SRF nº 96, de 26 de novembro de 1999","Ação Declaratória de Inconstitucionalidade – ADI nº 5.422"]}'

In [55]:
r2 = '{"deduplicated-references":["AD PGFN nº 3 de 18 de setembro de 2008","ADI RFB nº 12, de 2016","Acordo sobre o Novo Banco de Desenvolvimento","Anexo ao Decreto nº 361, de 10 de dezembro de 1991","Ato Declaratório Cosar nº 47, de 27 de novembro de 2000","Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019","Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024","Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005","Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016","Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007","Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016","Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004","Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004","Ato Declaratório Interpretativo SRF nº 26, de 26 de dezembro de 2003","Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007","Ato Declaratório Interpretativo SRF nº 8, de 23 de abril de 2003","Ato Declaratório Interpretativo nº 16, de 22 de dezembro de 2005","Ato Declaratório Normativo CST nº 10, de 1991","Ato Declaratório Normativo CST nº 11, de 1978","Ato Declaratório Normativo CST nº 16, de 27 de julho de 1979","Ato Declaratório Normativo CST nº 25, de 13 de dezembro de 1989","Ato Declaratório Normativo CST nº 29, de 25 de junho de 1986","Ato Declaratório Normativo Cosit nº 19, de 25 de outubro de 2000","Ato Declaratório Normativo Cosit nº 20, de 1989","Ato Declaratório Normativo Cosit nº 28, de 27 de dezembro de 1996","Ato Declaratório Normativo Cosit nº 7, de 13 de janeiro de 1997","Ato Declaratório Normativo Cosit nº 9, de 1º de abril de 1999","Ato Declaratório PGFN nº 1, de 12 de março de 2018","Ato Declaratório PGFN nº 3, de 30 de março de 2016","Ato Declaratório PGFN nº 4, de 16 de novembro de 2006","Ato Declaratório PGFN nº 5, de 3 de maio de 2016","Ato Declaratório PGFN nº 9, de 20 de dezembro de 2011","Ato Declaratório SRF nº 22, de 30 de abril de 1997","Ato Declaratório SRF nº 28, de 26 de abril de 2000","Ato Declaratório SRF nº 3, de 7 de janeiro de 1999","Ato Declaratório SRF nº 48, de 27 de junho de 2000","Ato Declaratório SRF nº 84, de 6 de outubro de 1999","Ato Declaratório SRF nº 95, de 26 de novembro de 1999","Ato Declaratório SRF nº 96, de 26 de novembro de 1999","Ação Declaratória de Inconstitucionalidade – ADI nº 5.422"]}'

In [57]:
j1 = json.loads(r1)

In [58]:
j2 = json.loads(r2)

In [59]:
j1.keys()

dict_keys(['deduplicated-references'])

In [60]:
j1['deduplicated-references']

['AD PGFN nº 3 de 18 de setembro de 2008',
 'ADI RFB nº 12, de 2016',
 'Acordo sobre o Novo Banco de Desenvolvimento',
 'Anexo ao Decreto nº 361, de 10 de dezembro de 1991',
 'Ato Declaratório Cosar nº 47, de 27 de novembro de 2000',
 'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019',
 'Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024',
 'Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005',
 'Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016',
 'Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007',
 'Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016',
 'Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004',
 'Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007',
 'Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004',
 'Ato Declaratório Interpretativo SRF nº 26, de 26 de dezembro de 2003',
 'Ato Declaratório Interpretativo SRF nº 8, de 23 de abril de 2003',
 'At

In [61]:
j2['deduplicated-references']

['AD PGFN nº 3 de 18 de setembro de 2008',
 'ADI RFB nº 12, de 2016',
 'Acordo sobre o Novo Banco de Desenvolvimento',
 'Anexo ao Decreto nº 361, de 10 de dezembro de 1991',
 'Ato Declaratório Cosar nº 47, de 27 de novembro de 2000',
 'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019',
 'Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024',
 'Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005',
 'Ato Declaratório Interpretativo RFB nº 1, de 18 de janeiro de 2016',
 'Ato Declaratório Interpretativo RFB nº 18, de 6 de dezembro de 2007',
 'Ato Declaratório Interpretativo RFB nº 3, de 27 de abril de 2016',
 'Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004',
 'Ato Declaratório Interpretativo SRF nº 24, de 4 de outubro de 2004',
 'Ato Declaratório Interpretativo SRF nº 26, de 26 de dezembro de 2003',
 'Ato Declaratório Interpretativo SRF nº 2, de 27 de março de 2007',
 'Ato Declaratório Interpretativo SRF nº 8, de 23 de abril de 2003',
 'At

In [105]:
r3 = '{"deduplicated-references":["Parecer Normativo CST nº 1, de 4 de fevereiro de 1985", "Parecer Normativo CST nº 10, de 17 de agosto de 1992", "Parecer Normativo CST nº 122, de 8 de junho de 1974", "Parecer Normativo CST nº 129, de 13 de setembro de 1973", "Parecer Normativo CST nº 130, de 8 de julho de 1970", "Parecer Normativo CST nº 173, de 26 de setembro de 1974", "Parecer Normativo CST nº 179, de 1970", "Parecer Normativo CST nº 2, de 15 de janeiro de 1980", "Parecer Normativo CST nº 25, de 1976", "Parecer Normativo CST nº 250, de 15 de março de 1971", "Parecer Normativo CST nº 28, de 29 de dezembro de 1983", "Parecer Normativo CST nº 32, de 17 de agosto de 1981", "Parecer Normativo CST nº 36, de 30 de maio de 1977", "Parecer Normativo CST nº 38, de 1975", "Parecer Normativo CST nº 44, de 30 de junho de 1976", "Parecer Normativo CST nº 6, de 19 de fevereiro de 1986", "Parecer Normativo CST nº 60, de 20 de junho de 1978", "Parecer Normativo CST nº 62, de 31 de agosto de 1976", "Parecer Normativo CST nº 66, de 5 de setembro de 1986", "Parecer Normativo CST nº 68, de 14 de setembro de 1976", "Parecer Normativo CST nº 72, de 1979", "Parecer Normativo CST nº 8, de 1979", "Parecer Normativo CST nº 9, de 1º de julho de 1983", "Parecer Normativo CST nº 90, de 16 de outubro de 1978", "Parecer Normativo Cosit nº 1, de 17 de março de 1994", "Parecer Normativo Cosit nº 1, de 24 de setembro de 2002", "Parecer Normativo Cosit nº 1, de 8 de agosto de 1995", "Parecer Normativo Cosit nº 11, de 30 de setembro de 1992"]}'

In [107]:
j3 = json.loads(r3)

In [109]:
np.sort(j3['deduplicated-references']).shape

(28,)

In [110]:
np.sort(j3['deduplicated-references'])

array(['Parecer Normativo CST nº 1, de 4 de fevereiro de 1985',
       'Parecer Normativo CST nº 10, de 17 de agosto de 1992',
       'Parecer Normativo CST nº 122, de 8 de junho de 1974',
       'Parecer Normativo CST nº 129, de 13 de setembro de 1973',
       'Parecer Normativo CST nº 130, de 8 de julho de 1970',
       'Parecer Normativo CST nº 173, de 26 de setembro de 1974',
       'Parecer Normativo CST nº 179, de 1970',
       'Parecer Normativo CST nº 2, de 15 de janeiro de 1980',
       'Parecer Normativo CST nº 25, de 1976',
       'Parecer Normativo CST nº 250, de 15 de março de 1971',
       'Parecer Normativo CST nº 28, de 29 de dezembro de 1983',
       'Parecer Normativo CST nº 32, de 17 de agosto de 1981',
       'Parecer Normativo CST nº 36, de 30 de maio de 1977',
       'Parecer Normativo CST nº 38, de 1975',
       'Parecer Normativo CST nº 44, de 30 de junho de 1976',
       'Parecer Normativo CST nº 6, de 19 de fevereiro de 1986',
       'Parecer Normativo CST nº 

In [111]:
r4='''{"deduplicated-references":[
"Parecer Normativo CST nº 1, de 4 de fevereiro de 1985",
"Parecer Normativo CST nº 10, de 17 de agosto de 1992",
"Parecer Normativo CST nº 122, de 8 de junho de 1974",
"Parecer Normativo CST nº 129, de 13 de setembro de 1973",
"Parecer Normativo CST nº 130, de 8 de julho de 1970",
"Parecer Normativo CST nº 130, de 1975",
"Parecer Normativo CST nº 173, de 26 de setembro de 1974",
"Parecer Normativo CST nº 179, de 1970",
"Parecer Normativo CST nº 2, de 15 de janeiro de 1980",
"Parecer Normativo CST nº 25, de 1976",
"Parecer Normativo CST nº 250, de 15 de março de 1971",
"Parecer Normativo CST nº 28, de 29 de dezembro de 1983",
"Parecer Normativo CST nº 32, de 17 de agosto de 1981",
"Parecer Normativo CST nº 36, de 30 de maio de 1977",
"Parecer Normativo CST nº 38, de 1975",
"Parecer Normativo CST nº 44, de 30 de junho de 1976",
"Parecer Normativo CST nº 6, de 19 de fevereiro de 1986",
"Parecer Normativo CST nº 60, de 20 de junho de 1978",
"Parecer Normativo CST nº 62, de 31 de agosto de 1976",
"Parecer Normativo CST nº 66, de 5 de setembro de 1986",
"Parecer Normativo CST nº 68, de 14 de setembro de 1976",
"Parecer Normativo CST nº 72, de 1979",
"Parecer Normativo CST nº 8, de 1979",
"Parecer Normativo CST nº 9, de 1º de julho de 1983",
"Parecer Normativo CST nº 90, de 16 de outubro de 1978",
"Parecer Normativo Cosit nº 1, de 17 de março de 1994",
"Parecer Normativo Cosit nº 1, de 24 de setembro de 2002",
"Parecer Normativo Cosit nº 1, de 8 de agosto de 1995",
"Parecer Normativo Cosit nº 11, de 30 de setembro de 1992"
]}'''

In [113]:
j4 = json.loads(r4)

In [114]:
np.sort(j4['deduplicated-references']).shape

(29,)

In [115]:
np.sort(j4['deduplicated-references'])

array(['Parecer Normativo CST nº 1, de 4 de fevereiro de 1985',
       'Parecer Normativo CST nº 10, de 17 de agosto de 1992',
       'Parecer Normativo CST nº 122, de 8 de junho de 1974',
       'Parecer Normativo CST nº 129, de 13 de setembro de 1973',
       'Parecer Normativo CST nº 130, de 1975',
       'Parecer Normativo CST nº 130, de 8 de julho de 1970',
       'Parecer Normativo CST nº 173, de 26 de setembro de 1974',
       'Parecer Normativo CST nº 179, de 1970',
       'Parecer Normativo CST nº 2, de 15 de janeiro de 1980',
       'Parecer Normativo CST nº 25, de 1976',
       'Parecer Normativo CST nº 250, de 15 de março de 1971',
       'Parecer Normativo CST nº 28, de 29 de dezembro de 1983',
       'Parecer Normativo CST nº 32, de 17 de agosto de 1981',
       'Parecer Normativo CST nº 36, de 30 de maio de 1977',
       'Parecer Normativo CST nº 38, de 1975',
       'Parecer Normativo CST nº 44, de 30 de junho de 1976',
       'Parecer Normativo CST nº 6, de 19 de fever

In [138]:
r5 = '{"deduplicated-references":["Sistema de Recolhimento Mensal Obrigatório (Carnê-Leão)","Solução de Consulta Cosit nº 100, de 28 de setembro de 2020","Solução de Consulta Cosit nº 102, de 7 de abril de 2014","Solução de Consulta Cosit nº 105, de 7 de abril de 2014","Solução de Consulta Cosit nº 106, de 25 de março de 2019","Solução de Consulta Cosit nº 111, de 29 de junho de 2021","Solução de Consulta Cosit nº 114, de 28 de setembro de 2020","Solução de Consulta Cosit nº 115, de 29 de junho de 2021","Solução de Consulta Cosit nº 116, de 26 de março de 2019","Solução de Consulta Cosit nº 122, de 26 de março de 2019","Solução de Consulta Cosit nº 123, de 13 de setembro de 2021","Solução de Consulta Cosit nº 125, de 28 de maio de 2014","Solução de Consulta Cosit nº 134, de 14 de setembro de 2021","Solução de Consulta Cosit nº 135, de 1º de dezembro de 2020","Solução de Consulta Cosit nº 138, de 8 de dezembro de 2020","Solução de Consulta Cosit nº 14, de 16 de janeiro de 2017","Solução de Consulta Cosit nº 140, de 21 de setembro de 2021","Solução de Consulta Cosit nº 142, de 21 de setembro de 2021","Solução de Consulta Cosit nº 144, de 21 de setembro de 2021","Solução de Consulta Cosit nº 147, de 24 de setembro de 2018","Solução de Consulta Cosit nº 15, de 29 de fevereiro de 2016","Solução de Consulta Cosit nº 152, de 26 de setembro de 2018","Solução de Consulta Cosit nº 157, de 24 de setembro de 2021","Solução de Consulta Cosit nº 159, de 7 de agosto de 2023","Solução de Consulta Cosit nº 161, de 7 de agosto de 2023","Solução de Consulta Cosit nº 162, de 7 de agosto de 2023","Solução de Consulta Cosit nº 164, de 26 de setembro de 2018","Solução de Consulta Cosit nº 166, de 28 de maio de 2019","Solução de Consulta Cosit nº 167, de 27 de setembro de 2021","Solução de Consulta Cosit nº 17, de 20 de abril de 2022","Solução de Consulta Cosit nº 171, de 27 de setembro de 2021","Solução de Consulta Cosit nº 173, de 3 de julho de 2015","Solução de Consulta Cosit nº 179, de 16 de agosto de 2023","Solução de Consulta Cosit nº 180, de 16 de agosto de 2023","Solução de Consulta Cosit nº 181, de 18 de agosto de 2023","Solução de Consulta Cosit nº 183, de 25 de junho de 2014","Solução de Consulta Cosit nº 19, de 18 de março de 2021","Solução de Consulta Cosit nº 19, de 4 de novembro de 2013","Solução de Consulta Cosit nº 194, de 5 de agosto de 2015","Solução de Consulta Cosit nº 200, de 14 de dezembro de 2021","Solução de Consulta Cosit nº 201, de 14 de junho de 2019","Solução de Consulta Cosit nº 202, de 30 de agosto de 2023","Solução de Consulta Cosit nº 204, de 4 de setembro de 2023","Solução de Consulta Cosit nº 204, de 5 de agosto de 2015","Solução de Consulta Cosit nº 206, de 24 de junho de 2019","Solução de Consulta Cosit nº 207, de 16 de novembro de 2018","Solução de Consulta Cosit nº 208, de 24 de abril de 2017","Solução de Consulta Cosit nº 209, de 16 de dezembro de 2021","Solução de Consulta Cosit nº 211, de 24 de junho de 2019","Solução de Consulta Cosit nº 214, de 20 de dezembro de 2021","Solução de Consulta Cosit nº 215, de 25 de junho de 2019","Solução de Consulta Cosit nº 217, de 21 de setembro de 2023","Solução de Consulta Cosit nº 218, de 21 de setembro de 2023","Solução de Consulta Cosit nº 221, de 22 de dezembro de 2021","Solução de Consulta Cosit nº 224, de 25 de setembro de 2023","Solução de Consulta Cosit nº 23, de 14 de junho de 2022","Solução de Consulta Cosit nº 231, de 9 de dezembro de 2015","Solução de Consulta Cosit nº 240, de 19 de agosto de 2019","Solução de Consulta Cosit nº 240, de 19 de maio de 2017","Solução de Consulta Cosit nº 256, de 15 de setembro de 2014","Solução de Consulta Cosit nº 261, de 26 de maio de 2017","Solução de Consulta Cosit nº 264, de 30 de outubro de 2023","Solução de Consulta Cosit nº 284, de 26 de dezembro de 2018","Solução de Consulta Cosit nº 309, de 26 de dezembro de 2018","Solução de Consulta Cosit nº 313, de 7 de novembro de 2014","Solução de Consulta Cosit nº 320, de 20 de junho de 2017","Solução de Consulta Cosit nº 325, de 17 de novembro de 2014","Solução de Consulta Cosit nº 33, de 26 de fevereiro de 2015","Solução de Consulta Cosit nº 337, de 28 de dezembro de 2018","Solução de Consulta Cosit nº 354, de 17 de dezembro de 2014","Solução de Consulta Cosit nº 356, de 17 de dezembro de 2014","Solução de Consulta Cosit nº 361, de 17 de dezembro de 2014","Solução de Consulta Cosit nº 4, de 3 de março de 2021","Solução de Consulta Cosit nº 42, de 14 de fevereiro de 2019","Solução de Consulta Cosit nº 48, de 24 de março de 2021","Solução de Consulta Cosit nº 50, de 22 de junho de 2020","Solução de Consulta Cosit nº 503, de 17 de outubro de 2017","Solução de Consulta Cosit nº 53, de 16 de dezembro de 2013","Solução de Consulta Cosit nº 6, de 3 de janeiro de 2019","Solução de Consulta Cosit nº 60, de 23 de junho de 2020","Solução de Consulta Cosit nº 600, de 21 de dezembro de 2017","Solução de Consulta Cosit nº 63, de 29 de março de 2021","Solução de Consulta Cosit nº 64, de 7 de março de 2014","Solução de Consulta Cosit nº 656, de 27 de dezembro de 2017","Solução de Consulta Cosit nº 66, de 23 de junho de 2020","Solução de Consulta Cosit nº 665, de 27 de dezembro de 2017","Solução de Consulta Cosit nº 674, de 27 de dezembro de 2017","Solução de Consulta Cosit nº 69, de 30 de dezembro de 2013","Solução de Consulta Cosit nº 7, de 13 de janeiro de 2017","Solução de Consulta Cosit nº 70, de 30 de dezembro de 2013","Solução de Consulta Cosit nº 72, de 30 de março de 2021","Solução de Consulta Cosit nº 73, de 31 de dezembro de 2013","Solução de Consulta Cosit nº 75, de 25 de junho de 2020","Solução de Consulta Cosit nº 76, de 3 de abril de 2023","Solução de Consulta Cosit nº 79, de 24 de março de 2015","Solução de Consulta Cosit nº 81, de 31 de março de 2014","Solução de Consulta Cosit nº 82, de 4 de abril de 2023","Solução de Consulta Cosit nº 92, de 29 de julho de 2020","Solução de Consulta Cosit nº 98, de 10 de maio de 2023","Solução de Consulta Interna Cosit nº 10, de 5 de junho de 2014","Solução de Consulta Interna Cosit nº 11, de 28 de junho de 2012","Solução de Consulta Interna Cosit nº 12, de 15 de maio de 2013","Solução de Consulta Interna Cosit nº 2, de 14 de janeiro de 2014","Solução de Consulta Interna Cosit nº 20, de 13 de agosto de 2013","Solução de Consulta Interna Cosit nº 23, de 30 de agosto de 2013","Solução de Consulta Interna Cosit nº 27, de 7 de julho de 2008","Solução de Consulta Interna Cosit nº 29, de 4 de novembro de 2013","Solução de Consulta Interna Cosit nº 3, de 8 de fevereiro de 2012","Solução de Consulta Interna Cosit nº 4, de 17 de abril de 2012","Solução de Consulta Interna Cosit nº 5, de 15 de fevereiro de 2013","Solução de Consulta Interna Cosit nº 5, de 28 de março de 2006","Solução de Consulta Interna Cosit nº 6, de 18 de maio de 2015","Solução de Consulta Interna Cosit nº 7, de 17 de maio de 2012","Solução de Consulta SRRF03/Disit nº 15, de 9 de março de 2009","Solução de Consulta SRRF06/Disit nº 6.007, de 25 de março de 2019","Solução de Consulta nº 140, de 5 de junho de 2015","Solução de Divergência Cosit nº 10, de 14 de agosto de 2014","Solução de Divergência Cosit nº 16, de 27 de setembro de 2012","Solução de Divergência Cosit nº 19, de 16 de setembro de 2013","Súmula Carf nº 42","Súmula nº 125 do Superior Tribunal de Justiça (STJ)","Súmula nº 136 do Superior Tribunal de Justiça (STJ)"]}'

In [139]:
j5 = json.loads(r5)

In [142]:
np.sort(j5['deduplicated-references'])

array(['Sistema de Recolhimento Mensal Obrigatório (Carnê-Leão)',
       'Solução de Consulta Cosit nº 100, de 28 de setembro de 2020',
       'Solução de Consulta Cosit nº 102, de 7 de abril de 2014',
       'Solução de Consulta Cosit nº 105, de 7 de abril de 2014',
       'Solução de Consulta Cosit nº 106, de 25 de março de 2019',
       'Solução de Consulta Cosit nº 111, de 29 de junho de 2021',
       'Solução de Consulta Cosit nº 114, de 28 de setembro de 2020',
       'Solução de Consulta Cosit nº 115, de 29 de junho de 2021',
       'Solução de Consulta Cosit nº 116, de 26 de março de 2019',
       'Solução de Consulta Cosit nº 122, de 26 de março de 2019',
       'Solução de Consulta Cosit nº 123, de 13 de setembro de 2021',
       'Solução de Consulta Cosit nº 125, de 28 de maio de 2014',
       'Solução de Consulta Cosit nº 134, de 14 de setembro de 2021',
       'Solução de Consulta Cosit nº 135, de 1º de dezembro de 2020',
       'Solução de Consulta Cosit nº 138, de 8 de d